In [1]:
cd ..

/home/cody/RepDistiller


In [2]:
from anatome import anatome

In [3]:
import torch
import copy
import glob
import torch.optim as optim
import torch.nn as nn
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.metrics import classification_report

from src.dataloader import make_data_loader
from src.model_loader import load_model
from src.trainers import first_eval, class_report
from src.analytics import fix_state_dict, df_builder

from train_student import load_teacher

/home/cody/miniconda3/envs/RepDist/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/cody/miniconda3/envs/RepDist/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/cody/miniconda3/envs/RepDist/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/cody/miniconda3/envs/RepDist/lib/python3

In [4]:
torch.manual_seed(42)
np.random.seed(42)
from dataset.cifar100 import get_cifar100_dataloaders

In [5]:
from models import model_dict

In [6]:
%%timeit
class_names = [['beaver', 'dolphin', 'otter', 'seal', 'whale'],
              ['aquarium_fish', 'flatfish', 'ray', 'shark', 'trout'],
              ['orchid', 'poppy', 'rose', 'sunflower', 'tulip'],
              ['bottle', 'bowl', 'can', 'cup', 'plate'],
              ['apple', 'mushroom', 'orange', 'pear', 'sweet_pepper'],
              ['clock', 'keyboard', 'lamp', 'telephone', 'television'],
              ['bed', 'chair', 'couch', 'table', 'wardrobe'],
              ['bee', 'beetle', 'butterfly', 'caterpillar', 'cockroach'],
              ['bear', 'leopard', 'lion', 'tiger', 'wolf'],
              ['bridge', 'castle', 'house', 'road', 'skyscraper'],
              ['cloud', 'forest', 'mountain', 'plain', 'sea'],
              ['camel', 'cattle', 'chimpanzee', 'elephant', 'kangaroo'],
              ['fox', 'porcupine', 'possum', 'raccoon', 'skunk'],
              ['crab', 'lobster', 'snail', 'spider', 'worm'],
              ['baby', 'boy', 'girl', 'man', 'woman'],
              ['crocodile', 'dinosaur', 'lizard', 'snake', 'turtle'],
              ['hamster', 'mouse', 'rabbit', 'shrew', 'squirrel'],
              ['maple_tree', 'oak_tree', 'palm_tree', 'pine_tree', 'willow_tree'],
              ['bicycle', 'bus', 'motorcycle', 'pickup_truck', 'train'],
              ['lawn_mower', 'rocket', 'streetcar', 'tank', 'tractor']]

class_names = list(np.concatenate(class_names).flat)

411 µs ± 41.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
student = model_dict['resnet32x4'](num_classes=100)
teacher = model_dict['resnet32x4'](num_classes=100)

In [8]:
teacher_path = './save/models/resnet32x4_vanilla/ckpt_epoch_240.pth'
teacher = load_teacher(teacher_path, 100)

==> loading teacher model
==> done


In [9]:
train_loader, val_loader = get_cifar100_dataloaders(200, 8, seq=True)
dataloaders = {'train': train_loader, 'val': val_loader}
dataset_sizes = {'train': 50000, 'val': 10000}

Files already downloaded and verified


/home/cody/miniconda3/envs/RepDist/lib/python3.7/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Files already downloaded and verified


/home/cody/miniconda3/envs/RepDist/lib/python3.7/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [10]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
student = student.to(device)
teacher = teacher.to(device)

In [11]:
sim_layers = ['layer1.4', 'layer2.4', 'layer3.4', 'fc']
#sim_layers = ['layer3.4']

In [12]:
teacher_hooks = anatome.SimilarityHook.create_hooks(teacher, sim_layers, 'svcca')
student_hooks = anatome.SimilarityHook.create_hooks(student, sim_layers , 'svcca')

In [13]:
sims = {}

In [14]:
%%time
import gc 

crd_45 = glob.glob('save/student_model/*crd*ts:0.4*lr:0.05*/*best.pth')
crd_30 = glob.glob('save/student_model/*crd*ts:0.3*lr:0.05*/*best.pth')
crd_60 = glob.glob('save/student_model/*crd*ts:0.6*lr:0.05*/*best.pth')
crd_75 = glob.glob('save/student_model/*crd*ts:0.7*lr:0.05*/*best.pth')
crd_90 = glob.glob('save/student_model/*crd*ts:0.9*lr:0.05*/*best.pth')

results = []

for sparsity in [crd_30, crd_45, crd_60, crd_75, crd_90]:
    
    d = np.array([0.0, 0.0, 0.0, 0.0])
    for model in sparsity:

        #make sure everything is clean
        [hook.clear() for hook in teacher_hooks]
        [hook.clear() for hook in student_hooks]
#         for hook in student_hooks:
#             del hook
        torch.cuda.empty_cache()
        gc.collect()

        #load student
        ckpt = torch.load(model, map_location=device)
        ckpt = fix_state_dict(ckpt['model'])
        student.load_state_dict(ckpt, strict=True)
        student = student.to(device)


        #add student hooks
        #student_hooks = anatome.SimilarityHook.create_hooks(student, sim_layers , 'svcca')

        print(f"iterating dataset for model: {model}")
        #make iterator and go through as much data as will fit in memory

        dset = iter(dataloaders['val'])
        for i in range(64):
            data = next(dset)
            with torch.no_grad():
                data = data[0].cuda()
                teacher(data)
                student(data)

        print(f'calculating distance for: {model}')
        distances = []
        for t_h, s_h in zip(teacher_hooks, student_hooks):
            distances.append(t_h.distance(s_h, size=16))
        d += np.array(distances)

    results.append(d / len(sparsity))
    print(results)
    
sims['crd_mag'] = results

iterating dataset for model: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_crd_r:1_a:0.0_b:0.8_3_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
calculating distance for: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_crd_r:1_a:0.0_b:0.8_3_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth


/home/cody/miniconda3/envs/RepDist/lib/python3.7/site-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bicubic is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


iterating dataset for model: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_crd_r:1_a:0.0_b:0.8_0_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
calculating distance for: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_crd_r:1_a:0.0_b:0.8_0_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
iterating dataset for model: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_crd_r:1_a:0.0_b:0.8_1_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
calculating distance for: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_crd_r:1_a:0.0_b:0.8_1_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
iterating dataset for model: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_crd_r:1_a:0.0_b:0.8_2_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
calculating distance for: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_crd_r:1_a:0.0_b:0.8_2_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
[array([0.11423176, 0.24394627, 0.53166722, 0.16832384])]
iterating dataset for model: save/stude

In [16]:
%%time
import gc 

all_45 = glob.glob('save/student_model/*resnet32x4*all*ts:0.4*lr:0.05/*best.pth')
all_30 = glob.glob('save/student_model/*resnet32x4*all*ts:0.3*lr:0.05/*best.pth')
all_60 = glob.glob('save/student_model/*resnet32x4*all*ts:0.6*lr:0.05/*best.pth')
all_75 = glob.glob('save/student_model/*resnet32x4*all*ts:0.7*lr:0.05/*best.pth')
all_90 = glob.glob('save/student_model/*resnet32x4*all*ts:0.9*lr:0.05/*best.pth')



results = []

for sparsity in [all_30, all_45, all_60, all_75, all_90]:
    
    d = np.array([0.0, 0.0, 0.0, 0.0])
    for model in sparsity:

        #make sure everything is clean
        [hook.clear() for hook in teacher_hooks]
        [hook.clear() for hook in student_hooks]
#         for hook in student_hooks:
#             del hook
        torch.cuda.empty_cache()
        gc.collect()

        #load student
        ckpt = torch.load(model, map_location=device)
        ckpt = fix_state_dict(ckpt['model'])
        student.load_state_dict(ckpt, strict=True)
        student = student.to(device)


        #add student hooks
        #student_hooks = anatome.SimilarityHook.create_hooks(student, sim_layers , 'svcca')

        print(f"iterating dataset for model: {model}")
        #make iterator and go through as much data as will fit in memory

        dset = iter(dataloaders['val'])
        for i in range(64):
            data = next(dset)
            with torch.no_grad():
                data = data[0].cuda()
                teacher(data)
                student(data)

        print(f'calculating distance for: {model}')
        distances = []
        for t_h, s_h in zip(teacher_hooks, student_hooks):
            distances.append(t_h.distance(s_h, size=16))
        d += np.array(distances)

    results.append(d / len(sparsity))
    print(results)
    
sims['crd_mag'] = results

iterating dataset for model: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_all_r:1.0_a:0.0_b:1.0_0_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth


/home/cody/miniconda3/envs/RepDist/lib/python3.7/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


calculating distance for: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_all_r:1.0_a:0.0_b:1.0_0_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth


/home/cody/miniconda3/envs/RepDist/lib/python3.7/site-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bicubic is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


iterating dataset for model: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_all_r:1.0_a:0.0_b:1.0_1_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
calculating distance for: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_all_r:1.0_a:0.0_b:1.0_1_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
iterating dataset for model: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_all_r:0.01_a:0.0_b:1.0_2_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
calculating distance for: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_all_r:0.01_a:0.0_b:1.0_2_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
iterating dataset for model: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_all_r:0.001_a:0.0_b:1.0_3_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
calculating distance for: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_all_r:0.001_a:0.0_b:1.0_3_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
iterating dataset for model: save/student_model/S:resnet32x4_T:resnet32x4_cifar

In [18]:
results

[array([0.02112578, 0.08427027, 0.38707027, 0.11473299]),
 array([0.04520613, 0.16413589, 0.50431585, 0.16410742]),
 array([0.08614644, 0.23635338, 0.56550932, 0.21078132]),
 array([0.1624126 , 0.34490173, 0.61096752, 0.27234997]),
 array([0.22337134, 0.4177568 , 0.59568648, 0.36828532])]

In [13]:
%%time
import gc 

mag_30 = glob.glob('save/student_model/*lr_0.05*ts:0.3*/*best.pth')
mag_45 = glob.glob('save/student_model/*lr_0.05*ts:0.4*/*best.pth') 
mag_60 = glob.glob('save/student_model/*lr_0.05*ts:0.6*/*best.pth') 
mag_75 = glob.glob('save/student_model/*lr_0.05*ts:0.7*/*best.pth')
mag_90 = glob.glob('save/student_model/*lr_0.05*ts:0.9*/*best.pth') 

results = []

for sparsity in [mag_30, mag_45, mag_60, mag_75, mag_90]:
    
    d = np.array([0.0, 0.0, 0.0, 0.0])
    for model in sparsity:

        #make sure everything is clean
        [hook.clear() for hook in teacher_hooks]
        [hook.clear() for hook in student_hooks]
#         for hook in student_hooks:
#             del hook
        torch.cuda.empty_cache()
        gc.collect()

        #load student
        ckpt = torch.load(model, map_location=device)
        ckpt = fix_state_dict(ckpt['model'])
        student.load_state_dict(ckpt, strict=True)
        student = student.to(device)


        #add student hooks
        #student_hooks = anatome.SimilarityHook.create_hooks(student, sim_layers , 'svcca')

        print(f"iterating dataset for model: {model}")
        #make iterator and go through as much data as will fit in memory

        dset = iter(dataloaders['val'])
        for i in range(64):
            data = next(dset)
            with torch.no_grad():
                data = data[0].cuda()
                teacher(data)
                student(data)

        print(f'calculating distance for: {model}')
        distances = []
        for t_h, s_h in zip(teacher_hooks, student_hooks):
            distances.append(t_h.distance(s_h, size=16))
        d += np.array(distances)

    results.append(d / len(sparsity))
    print(results)
    
sims['struct_mag'] = results

iterating dataset for model: save/student_model/resnet32x4_cifar100_lr_0.05_decay_0.0005_trial_0_ts:0.3_strat:struct/resnet32x4_best.pth
calculating distance for: save/student_model/resnet32x4_cifar100_lr_0.05_decay_0.0005_trial_0_ts:0.3_strat:struct/resnet32x4_best.pth


/home/cody/miniconda3/envs/RepDist/lib/python3.7/site-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bicubic is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


iterating dataset for model: save/student_model/resnet32x4_cifar100_lr_0.05_decay_0.0005_trial_1_ts:0.3_strat:struct/resnet32x4_best.pth
calculating distance for: save/student_model/resnet32x4_cifar100_lr_0.05_decay_0.0005_trial_1_ts:0.3_strat:struct/resnet32x4_best.pth
iterating dataset for model: save/student_model/resnet32x4_cifar100_lr_0.05_decay_0.0005_trial_3_ts:0.3_strat:struct/resnet32x4_best.pth
calculating distance for: save/student_model/resnet32x4_cifar100_lr_0.05_decay_0.0005_trial_3_ts:0.3_strat:struct/resnet32x4_best.pth
iterating dataset for model: save/student_model/resnet32x4_cifar100_lr_0.05_decay_0.0005_trial_2_ts:0.3_strat:struct/resnet32x4_best.pth
calculating distance for: save/student_model/resnet32x4_cifar100_lr_0.05_decay_0.0005_trial_2_ts:0.3_strat:struct/resnet32x4_best.pth
[array([0.11076383, 0.23093504, 0.55483049, 0.20734002])]
iterating dataset for model: save/student_model/resnet32x4_cifar100_lr_0.05_decay_0.0005_trial_0_ts:0.45_strat:struct/resnet32x4_

In [17]:
sim_30 = glob.glob('save/student_model/*similarity*ts:0.3*/*best.pth') 
sim_45 = glob.glob('save/student_model/*similarity*ts:0.4*/*best.pth')
sim_60 = glob.glob('save/student_model/*similarity*ts:0.6*/*best.pth')
sim_75 = glob.glob('save/student_model/*similarity*ts:0.7*/*best.pth') 
sim_90 = glob.glob('save/student_model/*similarity*ts:0.9*/*best.pth')

results = []

for sparsity in [sim_30, sim_45, sim_60, sim_75, sim_90]:
    
    d = np.array([0.0, 0.0, 0.0, 0.0])
    for model in sparsity:

        #make sure everything is clean
        [hook.clear() for hook in teacher_hooks]
        [hook.clear() for hook in student_hooks]
#         for hook in student_hooks:
#             del hook
        torch.cuda.empty_cache()
        gc.collect()

        #load student
        ckpt = torch.load(model, map_location=device)
        ckpt = fix_state_dict(ckpt['model'])
        student.load_state_dict(ckpt, strict=True)
        student = student.to(device)


        #add student hooks
        #student_hooks = anatome.SimilarityHook.create_hooks(student, sim_layers , 'svcca')

        print(f"iterating dataset for model: {model}")
        #make iterator and go through as much data as will fit in memory

        dset = iter(dataloaders['val'])
        for i in range(64):
            data = next(dset)
            with torch.no_grad():
                data = data[0].cuda()
                teacher(data)
                student(data)

        print(f'calculating distance for: {model}')
        distances = []
        for t_h, s_h in zip(teacher_hooks, student_hooks):
            distances.append(t_h.distance(s_h, size=16))
        d += np.array(distances)

    results.append(d / len(sparsity))
    print(results)

sims['sim'] = results

iterating dataset for model: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_similarity_r:1_a:0.0_b:3000.0_1_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
calculating distance for: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_similarity_r:1_a:0.0_b:3000.0_1_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
iterating dataset for model: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_similarity_r:1_a:0.0_b:3000.0_2_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
calculating distance for: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_similarity_r:1_a:0.0_b:3000.0_2_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
iterating dataset for model: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_similarity_r:1_a:0.0_b:3000.0_0_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
calculating distance for: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_similarity_r:1_a:0.0_b:3000.0_0_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
iterating dataset for model: save/stu

In [18]:
kd_30 = glob.glob('save/student_model/*resnet32x4*kd*ts:0.3*lr:0.05/*best.pth') 
kd_45 = glob.glob('save/student_model/*resnet32x4*kd*ts:0.4*lr:0.05/*best.pth') 
kd_60 = glob.glob('save/student_model/*resnet32x4*kd*ts:0.6*lr:0.05/*best.pth') 
kd_75 = glob.glob('save/student_model/*resnet32x4*kd*ts:0.7*lr:0.05/*best.pth') 
kd_90 = glob.glob('save/student_model/*resnet32x4*kd*ts:0.9*lr:0.05/*best.pth') 

results = []

for sparsity in [kd_30, kd_45, kd_60, kd_75, kd_90]:
    
    d = np.array([0.0, 0.0, 0.0, 0.0])
    for model in sparsity:

        #make sure everything is clean
        [hook.clear() for hook in teacher_hooks]
        [hook.clear() for hook in student_hooks]
#         for hook in student_hooks:
#             del hook
        torch.cuda.empty_cache()
        gc.collect()

        #load student
        ckpt = torch.load(model, map_location=device)
        ckpt = fix_state_dict(ckpt['model'])
        student.load_state_dict(ckpt, strict=True)
        student = student.to(device)


        #add student hooks
        #student_hooks = anatome.SimilarityHook.create_hooks(student, sim_layers , 'svcca')

        print(f"iterating dataset for model: {model}")
        #make iterator and go through as much data as will fit in memory

        dset = iter(dataloaders['val'])
        for i in range(64):
            data = next(dset)
            with torch.no_grad():
                data = data[0].cuda()
                teacher(data)
                student(data)

        print(f'calculating distance for: {model}')
        distances = []
        for t_h, s_h in zip(teacher_hooks, student_hooks):
            distances.append(t_h.distance(s_h, size=16))
        d += np.array(distances)

    results.append(d / len(sparsity))
    print(results)

sims['kd'] = results

iterating dataset for model: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_kd_r:0.1_a:0.9_b:0.0_2_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
calculating distance for: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_kd_r:0.1_a:0.9_b:0.0_2_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
iterating dataset for model: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_kd_r:0.1_a:0.9_b:0.0_4_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
calculating distance for: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_kd_r:0.1_a:0.9_b:0.0_4_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
iterating dataset for model: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_kd_r:0.1_a:0.9_b:0.0_1_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
calculating distance for: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_kd_r:0.1_a:0.9_b:0.0_1_ts:0.3_strat:struct_lr:0.05/resnet8x4_best.pth
iterating dataset for model: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_kd_r:0.1

In [19]:
at_45 = glob.glob('save/student_model/*resnet32x4*attention*ts:0.4*lr:0.005/*best.pth')
at_30 = glob.glob('save/student_model/*resnet32x4*attention*ts:0.3*lr:0.005/*best.pth')
at_60 = glob.glob('save/student_model/*resnet32x4*attention*ts:0.6*lr:0.005/*best.pth')
at_75 = glob.glob('save/student_model/*resnet32x4*attention*ts:0.7*lr:0.005/*best.pth')
at_90 = glob.glob('save/student_model/*resnet32x4*attention*ts:0.9*lr:0.005/*best.pth')

results = []

for sparsity in [at_30, at_45, at_60, at_75, at_90]:
    
    d = np.array([0.0, 0.0, 0.0, 0.0])
    for model in sparsity:

        #make sure everything is clean
        [hook.clear() for hook in teacher_hooks]
        [hook.clear() for hook in student_hooks]
#         for hook in student_hooks:
#             del hook
        torch.cuda.empty_cache()
        gc.collect()

        #load student
        ckpt = torch.load(model, map_location=device)
        ckpt = fix_state_dict(ckpt['model'])
        student.load_state_dict(ckpt, strict=True)
        student = student.to(device)


        #add student hooks
        #student_hooks = anatome.SimilarityHook.create_hooks(student, sim_layers , 'svcca')

        print(f"iterating dataset for model: {model}")
        #make iterator and go through as much data as will fit in memory

        dset = iter(dataloaders['val'])
        for i in range(64):
            data = next(dset)
            with torch.no_grad():
                data = data[0].cuda()
                teacher(data)
                student(data)

        print(f'calculating distance for: {model}')
        distances = []
        for t_h, s_h in zip(teacher_hooks, student_hooks):
            distances.append(t_h.distance(s_h, size=16))
        d += np.array(distances)

    results.append(d / len(sparsity))
    print(results)

sims['at'] = results

iterating dataset for model: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_attention_r:1_a:0.0_b:1000.0_3_ts:0.3_strat:struct_lr:0.005/resnet8_best.pth
calculating distance for: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_attention_r:1_a:0.0_b:1000.0_3_ts:0.3_strat:struct_lr:0.005/resnet8_best.pth
iterating dataset for model: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_attention_r:1_a:0.0_b:1000.0_4_ts:0.3_strat:struct_lr:0.005/resnet8_best.pth
calculating distance for: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_attention_r:1_a:0.0_b:1000.0_4_ts:0.3_strat:struct_lr:0.005/resnet8_best.pth
iterating dataset for model: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_attention_r:1_a:0.0_b:1000.0_2_ts:0.3_strat:struct_lr:0.005/resnet8_best.pth
calculating distance for: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_attention_r:1_a:0.0_b:1000.0_2_ts:0.3_strat:struct_lr:0.005/resnet8_best.pth
iterating dataset for model: save/student_model/S

In [20]:
pkt_45 = glob.glob('save/student_model/*resnet32x4*pkt*ts:0.4*lr:0.005/*best.pth')
pkt_30 = glob.glob('save/student_model/*resnet32x4*pkt*ts:0.3*lr:0.005/*best.pth')
pkt_60 = glob.glob('save/student_model/*resnet32x4*pkt*ts:0.6*lr:0.005/*best.pth')
pkt_75 = glob.glob('save/student_model/*resnet32x4*pkt*ts:0.7*lr:0.005/*best.pth')
pkt_90 = glob.glob('save/student_model/*resnet32x4*pkt*ts:0.9*lr:0.005/*best.pth')

results = []

for sparsity in [pkt_30, pkt_45, pkt_60, pkt_75, pkt_90]:
    
    d = np.array([0.0, 0.0, 0.0, 0.0])
    for model in sparsity:

        #make sure everything is clean
        [hook.clear() for hook in teacher_hooks]
        [hook.clear() for hook in student_hooks]
#         for hook in student_hooks:
#             del hook
        torch.cuda.empty_cache()
        gc.collect()

        #load student
        ckpt = torch.load(model, map_location=device)
        ckpt = fix_state_dict(ckpt['model'])
        student.load_state_dict(ckpt, strict=True)
        student = student.to(device)


        #add student hooks
        #student_hooks = anatome.SimilarityHook.create_hooks(student, sim_layers , 'svcca')

        print(f"iterating dataset for model: {model}")
        #make iterator and go through as much data as will fit in memory

        dset = iter(dataloaders['val'])
        for i in range(64):
            data = next(dset)
            with torch.no_grad():
                data = data[0].cuda()
                teacher(data)
                student(data)

        print(f'calculating distance for: {model}')
        distances = []
        for t_h, s_h in zip(teacher_hooks, student_hooks):
            distances.append(t_h.distance(s_h, size=16))
        d += np.array(distances)

    results.append(d / len(sparsity))
    print(results)

sims['pkt'] = results

iterating dataset for model: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_pkt_r:1_a:0.0_b:30000.0_3_ts:0.3_strat:struct_lr:0.005/resnet8x4_best.pth
calculating distance for: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_pkt_r:1_a:0.0_b:30000.0_3_ts:0.3_strat:struct_lr:0.005/resnet8x4_best.pth
iterating dataset for model: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_pkt_r:1_a:0.0_b:30000.0_2_ts:0.3_strat:struct_lr:0.005/resnet8x4_best.pth
calculating distance for: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_pkt_r:1_a:0.0_b:30000.0_2_ts:0.3_strat:struct_lr:0.005/resnet8x4_best.pth
iterating dataset for model: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_pkt_r:1_a:0.0_b:30000.0_0_ts:0.3_strat:struct_lr:0.005/resnet8x4_best.pth
calculating distance for: save/student_model/S:resnet32x4_T:resnet32x4_cifar100_pkt_r:1_a:0.0_b:30000.0_0_ts:0.3_strat:struct_lr:0.005/resnet8x4_best.pth
iterating dataset for model: save/student_model/S:resnet32x4_T:resn

In [21]:
sims

{'struct_mag': [array([0.11076383, 0.23093504, 0.55483049, 0.20734002]),
  array([0.12564581, 0.26617607, 0.57405552, 0.23733528]),
  array([0.16858421, 0.31926581, 0.60245958, 0.273138  ]),
  array([0.22644553, 0.39126544, 0.63017884, 0.28000446]),
  array([0.23203238, 0.42617299, 0.62021048, 0.31071572])],
 'sim': [array([0.07492287, 0.16262624, 0.44531006, 0.1222202 ]),
  array([0.11477873, 0.2398724 , 0.53158841, 0.16901009]),
  array([0.16076702, 0.30538747, 0.58111922, 0.21462908]),
  array([0.22093787, 0.38569866, 0.62026973, 0.22960122]),
  array([0.22687929, 0.4183962 , 0.61449039, 0.34005357])],
 'kd': [array([0.0793261 , 0.17867119, 0.48005776, 0.1256899 ]),
  array([0.10877489, 0.23096418, 0.53844362, 0.15449415]),
  array([0.15264054, 0.30507977, 0.58796684, 0.19709052]),
  array([0.22057854, 0.39440924, 0.62250158, 0.22471598]),
  array([0.24369474, 0.43213832, 0.59883173, 0.42920731])],
 'at': [array([0.01790905, 0.04661237, 0.36544695, 0.09287789]),
  array([0.04311327,

results with adaptive_avg_pool2d
[array([0.11076383, 0.23093504, 0.5568569 , 0.20734002])]